<a href="https://colab.research.google.com/github/HatemHassanMohamed/Generative-AI-Tasks-subProjects/blob/main/Bert(Classification%2C_Summarization%2C_translation%2CQ%26A...).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch datasets

In [2]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import dataloader

# Text clssification

In [4]:
dataset = load_dataset('imdb')

train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
train_texts = list(train_texts)
train_labels = list(train_labels)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True,max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True,max_length=512)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)

In [9]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [10]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: watcherv543 (watcherv543-ess) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.682100
20,0.699300
30,0.685500
40,0.686000
50,0.691300
60,0.682800
70,0.695300
80,0.685600
90,0.661200
100,0.634600


TrainOutput(global_step=2500, training_loss=0.33135624063014985, metrics={'train_runtime': 1057.6894, 'train_samples_per_second': 18.909, 'train_steps_per_second': 2.364, 'total_flos': 2649347973120000.0, 'train_loss': 0.33135624063014985, 'epoch': 1.0})

In [11]:
model.save_pretrained('./distilbert-sentiment')

In [12]:
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained('./distilbert-sentiment')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

test_text = "I really loved the movie!"
test_encoding = tokenizer(test_text, truncation=True, padding=True, return_tensors="pt")
test_encoding = {k: v.to(device) for k, v in test_encoding.items()}

with torch.no_grad():
    outputs = model(**test_encoding)

prediction = torch.argmax(outputs.logits).item()
print(f"Predicted sentiment: {'positive' if prediction == 1 else 'negative'}")

Predicted sentiment: positive


# Summarization

In [13]:
from transformers import BartTokenizer, BartForConditionalGeneration

In [14]:
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

text = """The BERT model has transformed the field of Natural Language Processing by introducing a powerful bidirectional language model.
It enables a deeper understanding of language context and has been successfully applied to various tasks including question answering,
text classification, and language translation."""

inputs = tokenizer([text], max_length=1024, return_tensors='pt', truncation=True)
summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=100, early_stopping=True)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

The BERT model has transformed the field of Natural Language Processing by introducing a powerful bidirectional language model. It enables a deeper understanding of language context and has been successfully applied to various tasks including question answering,text classification, and language translation. It has been used to develop the BERT language model in the U.S.


# Translation With mBart


In [15]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

tokenizer.src_lang = "en_XX"
encoded_text = tokenizer(text, return_tensors="pt", truncation=True)

generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"])
translation = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
print(f"Translation: {translation}")

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Translation: Le modèle BERT a transformé le domaine du traitement des langues naturelles en introduisant un puissant modèle linguistique bidirectionnel qui permet une meilleure compréhension du contexte linguistique et qui a été appliqué avec succès à diverses tâches, notamment à la réponse aux questions, à la classification des textes et à la traduction linguistique.


# Q&A

In [17]:
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
import torch
from transformers import pipeline

model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForQuestionAnswering.from_pretrained(model_name)

context = """
BERT, or Bidirectional Encoder Representations from Transformers, is a transformer-based machine learning technique for natural language processing (NLP) pre-training developed by Google. BERT was created and published in 2018 by Jacob Devlin and his colleagues from Google.
"""
question = "Who developed BERT?"

inputs = tokenizer(question, context, return_tensors='pt', truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = torch.argmax(outputs.start_logits)
answer_end_index = torch.argmax(outputs.end_logits) + 1
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start_index:answer_end_index]))

print(f"Question: {question}")
print(f"Answer: {answer}")


Question: Who developed BERT?
Answer: google
